In [98]:
import gc

import matplotlib.pyplot as plt
import numpy as np

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import segmentation_models_pytorch as smp

from src.models.BaselineModel import BaselineModel
from src.evaluation.evaluate_result import evaluate_result
from src.datasets.INRIAAerialImageLabellingDataset import (
    INRIAAerialImageLabellingDataset,
)

from src.datasets.utils.ResizeToDivisibleBy32 import ResizeToDivisibleBy32

In [2]:
torch.cuda.is_available()

True

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
VAL_SIZE = 0.2
BATCH_SIZE = 1
SEED = 42
INRIA_DATASET_PATH = "data/INRIAAerialImageLabellingDataset"

In [6]:
labeled_dataset = INRIAAerialImageLabellingDataset(
    INRIA_DATASET_PATH, transforms=[ResizeToDivisibleBy32()]
)
print(len(labeled_dataset))

data/INRIAAerialImageLabellingDataset\train
180


In [7]:
test_dataset = INRIAAerialImageLabellingDataset(
    INRIA_DATASET_PATH, split="test", transforms=[ResizeToDivisibleBy32()]
)
print(len(test_dataset))

data/INRIAAerialImageLabellingDataset\test
144


In [8]:
# train_indices, val_indices = train_test_split(
#     range(len(labeled_dataset)),
#     test_size=VAL_SIZE,
#     random_state=SEED
# )

# print(f"Number of training samples: {len(train_indices)}")
# print(f"Number of validation samples: {len(val_indices)}")

In [9]:
# train_split = Subset(labeled_dataset, train_indices)
# print(len(train_split))
# val_split = Subset(labeled_dataset, val_indices)
# print(len(val_split))

In [10]:
# train_loader = DataLoader(train_split, batch_size=BATCH_SIZE, shuffle=True)
# val_loader = DataLoader(val_split, batch_size=BATCH_SIZE, shuffle=True)

train_loader = DataLoader(labeled_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [11]:
for images, masks in train_loader:
    print(images.shape)
    print(masks.shape)
    break

torch.Size([1, 3, 5024, 5024])
torch.Size([1, 1, 5024, 5024])


In [19]:
to_pil_transform = transforms.ToPILImage()

In [12]:
img = to_pil_transform(images.squeeze())

In [13]:
img.show()

In [14]:
msk = to_pil_transform(masks.squeeze())

In [49]:
msk.show()

In [41]:
model = smp.Unet(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,  # model output channels (number of classes in your dataset)
).to(device)

In [99]:
baseline_model = BaselineModel()

In [42]:
torch.cuda.empty_cache()

In [100]:
model.eval()
with torch.no_grad():
    for images, masks in train_loader:
        print(images.shape)
        print(masks.shape)
        break
    output = model(images.to(device))
    output = (output > 0.5).float()

    output_baseline = baseline_model(images.to(device))
    output_baseline = (output_baseline > 0.5).float()

torch.Size([1, 3, 5024, 5024])
torch.Size([1, 1, 5024, 5024])


In [101]:
print(output.shape)

torch.Size([1, 1, 5024, 5024])


In [102]:
print(output_baseline.shape)

torch.Size([1, 1, 5024, 5024])


In [103]:
outp = to_pil_transform(output.squeeze().cpu().numpy())

In [104]:
unique, counts = np.unique(outp, return_counts=True)
print(dict(zip(unique, counts)))

{0: 25165645, 255: 74931}


In [105]:
unique, counts = np.unique(output.cpu(), return_counts=True)
print(dict(zip(unique, counts)))

{0.0: 25165645, 1.0: 74931}


In [106]:
outp.show()

In [107]:
rand_outp = to_pil_transform(output_baseline.squeeze().cpu().numpy())

In [108]:
unique, counts = np.unique(rand_outp, return_counts=True)
print(dict(zip(unique, counts)))

{0: 12625381, 255: 12615195}


In [109]:
rand_outp.show()

In [110]:
evaluate_result(output, masks)

{'IoU score': tensor(0.0026),
 'F1 score': tensor(0.0052),
 'Accuracy': tensor(0.7816),
 'Recall': tensor(0.0488)}

In [111]:
evaluate_result(output_baseline, masks)

{'IoU score': tensor(0.1780),
 'F1 score': tensor(0.3022),
 'Accuracy': tensor(0.5001),
 'Recall': tensor(0.5222)}

In [112]:
evaluate_result(masks, masks)

{'IoU score': tensor(1.),
 'F1 score': tensor(1.),
 'Accuracy': tensor(1.),
 'Recall': tensor(1.)}

In [ ]:
model.eval()

with torch.no_grad():
    for i, (images, masks) in enumerate(train_loader):

        cumulated_iou_score = 0.0
        height = images.shape[2]
        width = images.shape[3]

        # if height % 32 == 0 and width % 32 == 0:
        #     print("Height and width are divisible by 32")
        # else:
        #     print("Height and width are not divisible by 32")

        output = model(images.to(device))
        classes_output = (output > 0.5).float()
        # output = torch.softmax(output, dim=1)
        # output = torch.argmax(output, dim=1)
        # class_output = torch.sigmoid(output)
        # class_output = torch.round(output)

        print("output")
        print(type(output))
        print(output.shape)
        print(output.max())
        print(output.min())
        print()
        # print("masks")
        # print(type(masks))
        # print(masks.shape)
        # print(masks.max())
        # print(masks.min())
        # print()
        # print("class_output")
        # print(type(class_output))
        # print(class_output.shape)
        # print(class_output.max())
        # print(class_output.min())
        # print()

        # images = to_pil_transform(images.squeeze())
        # images.show()
        # images_array = np.array(images)

        # Show the images using matplotlib
        # plt.imshow(images_array, cmap='gray')
        # plt.axis('off')

        # output = to_pil_transform(output.squeeze())
        # output.show()
        # output_array = np.array(output.squeeze())

        # Show the output using matplotlib
        # plt.figure()
        # plt.imshow(output_array, cmap='gray')
        # plt.axis('off')
        # plt.show()

        # class_output_array = np.array(class_output.squeeze())

        # masks_pil= to_pil_transform(masks.squeeze())
        # masks.show()
        # Convert the PIL Image to a numpy array
        # mask_array = np.array(masks_pil)

        # Show the mask using matplotlib
        # plt.imshow(mask_array, cmap='gray')
        # plt.axis('off')
        # plt.show()

        tp, fp, fn, tn = smp.metrics.get_stats(
            class_output.squeeze().to("cpu"),
            masks.int().squeeze(),
            mode="binary",
            threshold=0.5,
            # output.squeeze().to("cpu"), masks.int().squeeze(), mode="binary", threshold=0.5
        )
        # print(f"TP: {tp}")
        # print(f"FP: {fp}")
        # print(f"TN: {tn}")
        # print(f"FN: {fn}")

        iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")
        print(f"IoU score: {iou_score}")

        cumulated_iou_score += iou_score
        del images, masks, output
        gc.collect()
        torch.cuda.empty_cache()

        if i >= 1:
            break

average_iou_score = cumulated_iou_score / i
print(f"Average IoU score: {average_iou_score}")

In [ ]:
plt.imshow(images_array, cmap="gray")
plt.axis("off")
plt.show()

In [ ]:
plt.imshow(mask_array, cmap="gray")
plt.axis("off")
plt.show()

In [ ]:
# plt.imshow(output_array, cmap='gray')
# plt.axis('off')
# plt.show()
plt.matshow(output_array[0, :, :])
plt.matshow(output_array[1, :, :])

In [ ]:
plt.imshow(class_output_array, cmap="gray")
plt.axis("off")
plt.show()

In [ ]:
plt.matshow(class_output_array)

In [ ]:
class_output_array

In [ ]:
class_output_array.max()

In [ ]:
class_output_array.min()

In [ ]:
unique, counts = np.unique(class_output_array, return_counts=True)
print(dict(zip(unique, counts)))